# 3. Užduotis

Duomenys iš pirmos užduoties 3) punkto, t.y. a.d. $X$, turinčio binominį skirstinį su
parametrais $k$ ir $p$, t. y.

$$
P(X = m | k, p) = C^k_m p^m {(1 − p)}^{k−m},
$$

dydžio $n = 50$ paprastoji atsitiktinė imtis.

In [7]:
set.seed(314)

n <- 50

k <- 20
p <- 0.4

observ <- rbinom(n, k, p)

Didžiausio tikėtinumo metodu rasime parametru įverčius:

In [8]:
loglik <- function(observations) {
    function(theta) {
        p <- theta[1]
     
        sum(
            log(choose(k, observations))
            + log(p) * observations
            + log(1 - p) * (k - observations)
        )
    }
}

In [9]:
maxlik_optimiser <- function(f, start) {
    suppressPackageStartupMessages(require(maxLik))
    mle <- maxLik::maxLik(logLik = f, start = start)
    r <- list(
        estimates = coef(mle),
        hessian = hessian(mle),
        gradient = gradient(mle)
    )
    return(r)
}

mle_estimator <- function(loglikelihood, optimiser) {
    function(data, start) {
        optimiser(loglikelihood(data), start)
    }
}

In [10]:
moment_estimator <- function(observations) {
    mean(observations) / length(observations)
}

(start_theta <- moment_estimator(observ))

[1] 0.1464

In [11]:
(mle <- mle_estimator(loglik, maxlik_optimiser)(observ, start_theta))

-4309.442


Raskite tikslų parametro $p$ pasikliovimo intervalą
naudodami beta skirstinio kvantilius.

In [12]:
confidence <- 0.95
alpha <- 1 - confidence
alpha / 2

[1] 0.025

Remsimės pavyzdžiu iš vadovėlio 141-142 psl.

In [15]:
complete_statistic <- sum(observ)
p_lo <- 1 - qbeta(1 - alpha/2, n * k - complete_statistic + 1, complete_statistic)
p_hi <- 1 - qbeta(alpha/2, n * k - complete_statistic, complete_statistic + 1) 
p_hat <- mle$estimates[1]
data.frame(p, p_lo, p_hat,p_hi)

p,p_lo,p_hat,p_hi
<dbl>,<dbl>,<dbl>,<dbl>
0.4,0.3360757,0.366,0.3967159
